In [1]:
import os

In [2]:
pwd

'c:\\Users\\rohitsun\\PycharmProjects\\text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\rohitsun\\PycharmProjects\\text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_files: Path
    unzip_dir: Path



In [6]:
from textSummarizer.constants import *
import sys
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir, 
                source_URL= config.source_URL,
                local_data_files = config.local_data_files,
                unzip_dir=config.unzip_dir
          )
          return data_ingestion_config


In [8]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_files
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size {get_size(Path(self.config.local_data_files))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [17]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-20 20:54:30,377: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-20 20:54:30,377: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-20 20:54:30,386: INFO: common: created directory at:artifacts]
[2024-04-20 20:54:30,388: INFO: common: created directory at:artifacts/data_ingestion]
[2024-04-20 20:54:32,213: INFO: 2582151979: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 4059296
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "213e17bc706b22154528557081d339aae1106ca95965ee0378f9d30414580f93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E740:2C94AA:14012F:1A5693:6623DD6F
Accept-Ranges: bytes
Date: Sat, 20 Apr 2024 15:24:31 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10244-MAA
X-Cache: HIT